In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import mutual_info_classif

In [2]:
df = pd.read_parquet("../data/processed/dados_tratados.parquet")
df.head()

,RA,Fase,Ano nasc,Gênero,Instituição de ensino,INDE,Nº Av,IAA,IEG,IPS,...,IDA,Matem,Portug,Inglês,Indicado,Atingiu PV,IPV,IAN,Defas,ano_base
0,RA-1,7,2003,Menina,Escola Pública,5.783,4.0,8.3,4.1,5.6,...,4.0,2.7,3.5,6.0,Sim,Não,7.278,5.0,-1,2022
1,RA-2,7,2005,Menina,Rede Decisão,7.055,4.0,8.8,5.2,6.3,...,6.8,6.3,4.5,9.7,Não,Não,6.778,10.0,0,2022
2,RA-3,7,2005,Menina,Rede Decisão,6.591,4.0,0.0,7.9,5.6,...,5.6,5.8,4.0,6.9,Não,Não,7.556,10.0,0,2022
3,RA-4,7,2005,Menino,Rede Decisão,5.951,4.0,8.8,4.5,5.6,...,5.0,2.8,3.5,8.7,Não,Não,5.278,10.0,0,2022
4,RA-5,7,2005,Menina,Rede Decisão,7.427,4.0,7.9,8.6,5.6,...,5.2,7.0,2.9,5.7,Não,Não,7.389,10.0,0,2022


In [3]:
print("Shape:", df.shape)

Shape: (3030, 21)


In [4]:
print("Tipos de dados:")
print(df.dtypes)

Tipos de dados:
RA                        object
Fase                      object
Ano nasc                   int64
Gênero                    object
Instituição de ensino     object
INDE                      object
Nº Av                    float64
IAA                      float64
IEG                      float64
IPS                      float64
Rec Psicologia            object
IDA                      float64
Matem                    float64
Portug                   float64
Inglês                   float64
Indicado                  object
Atingiu PV                object
IPV                      float64
IAN                      float64
Defas                      int64
ano_base                   int64
dtype: object


In [5]:
duplicados = df.duplicated(subset=["RA", "ano_base"]).sum()
print(f"\nDuplicados (RA, ano_base): {duplicados}")


Duplicados (RA, ano_base): 0


In [6]:
columns = df.columns.to_list()

for c in columns:
    unicos = df[c].nunique(dropna=False)
    print(f"Valores unicos em {c}: {unicos}")


Valores unicos em RA: 1661
Valores unicos em Fase: 88
Valores unicos em Ano nasc: 21
Valores unicos em Gênero: 4
Valores unicos em Instituição de ensino: 13
Valores unicos em INDE: 2710
Valores unicos em Nº Av: 7
Valores unicos em IAA: 71
Valores unicos em IEG: 805
Valores unicos em IPS: 58
Valores unicos em Rec Psicologia: 6
Valores unicos em IDA: 164
Valores unicos em Matem: 84
Valores unicos em Portug: 88
Valores unicos em Inglês: 85
Valores unicos em Indicado: 3
Valores unicos em Atingiu PV: 3
Valores unicos em IPV: 921
Valores unicos em IAN: 3
Valores unicos em Defas: 9
Valores unicos em ano_base: 3


In [7]:
df["Atingiu PV"].value_counts(dropna=False)

Atingiu PV
None    2170
Não      747
Sim      113
Name: count, dtype: int64

In [8]:
mapa_instituicao = {
    # Pública
    "Pública": "Publica",
    "Escola Pública": "Publica",

    # Privada
    "Privada": "Privada",

    # Privada com parcerias / bolsa
    "Privada - Programa de Apadrinhamento": "Privada_Parceria",
    "Privada - Programa de apadrinhamento": "Privada_Parceria",
    "Privada *Parcerias com Bolsa 100%": "Privada_Parceria",
    "Privada - Pagamento por *Empresa Parceira": "Privada_Parceria",

    # Redes específicas
    "Rede Decisão": "Rede_Decisao",

    # Situação acadêmica posterior
    "Concluiu o 3º EM": "Universitario",
    "Bolsista Universitário *Formado (a)": "Universitario",

    # Outros
    "Escola JP II": "Outros",
    "Nenhuma das opções acima": "Outros",
}

df["Instituicao_padronizada"] = (
    df["Instituição de ensino"]
      .map(mapa_instituicao)
      .fillna("Nao_informado")
)
df["Instituicao_padronizada"].value_counts(dropna=False)


Instituicao_padronizada
Publica             2474
Privada_Parceria     314
Rede_Decisao         106
Privada              104
Universitario         27
Outros                 4
Nao_informado          1
Name: count, dtype: int64

In [9]:
mapa_psicologia = {
    # Avaliado sem risco
    "Sem limitações": "Sem_Risco",

    # Em processo de avaliação
    "Requer avaliação": "Em_Avaliacao",
    "Não avaliado": "Em_Avaliacao",

    # Casos com atenção psicológica
    "Não atendido": "Risco_Psicologico",

    # Avaliado e não indicado
    "Não indicado": "Nao_Indicado"
}

df["Rec_Psicologia_padronizada"] = (
    df["Rec Psicologia"]
      .map(mapa_psicologia)
      .fillna("Nao_Informado")
)

# Verificação
df["Rec_Psicologia_padronizada"].value_counts(dropna=False)


Rec_Psicologia_padronizada
Nao_Informado        2170
Risco_Psicologico     406
Sem_Risco             254
Em_Avaliacao          163
Nao_Indicado           37
Name: count, dtype: int64

In [10]:
def classificar_defas(x):
    if x >= 0:
        return "Em fase"
    elif x >= -2:
        return "Moderada"
    else:
        return "Severa"

df["classe_defas"] = df["Defas"].apply(classificar_defas)
print("Distribuição da variável alvo:")
print(df["classe_defas"].value_counts(normalize=True))

Distribuição da variável alvo:
classe_defas
Moderada    0.541914
Em fase     0.443234
Severa      0.014851
Name: proportion, dtype: float64


In [11]:
missing = (
    df.isna()
      .mean()
      .sort_values(ascending=False)
      .reset_index()
      .rename(columns={"index": "feature", 0: "pct_missing"})
)

print("\nPercentual de valores ausentes:")
print(missing)


Percentual de valores ausentes:
                       feature  pct_missing
0               Rec Psicologia     0.716172
1                   Atingiu PV     0.716172
2                     Indicado     0.716172
3                       Inglês     0.639934
4                       Portug     0.061056
5                        Matem     0.060726
6                          IDA     0.058746
7                          IPV     0.058746
8                          IPS     0.056436
9                          IAA     0.054455
10                         IEG     0.025083
11                       Nº Av     0.025083
12       Instituição de ensino     0.000330
13                         IAN     0.000000
14                       Defas     0.000000
15                    ano_base     0.000000
16     Instituicao_padronizada     0.000000
17  Rec_Psicologia_padronizada     0.000000
18                          RA     0.000000
19                        Fase     0.000000
20                        INDE     0.000000

In [13]:
# Critério técnico:
# > 40% nulos  -> excluir
# 10% a 40%   -> avaliar
# < 10%       -> imputação simples

In [14]:
features_excluir_missing = missing.loc[
    missing["pct_missing"] > 0.40, "feature"
].tolist()
print("\nFeatures a excluir por alto percentual de missing:", features_excluir_missing)


Features a excluir por alto percentual de missing: ['Rec Psicologia', 'Atingiu PV', 'Indicado', 'Inglês']


In [16]:
features_missing = missing.loc[
    missing["pct_missing"] > 0.1, "feature"
].tolist()
print("\nFeatures a excluir por alto percentual de missing:", features_missing)


Features a excluir por alto percentual de missing: ['Rec Psicologia', 'Atingiu PV', 'Indicado', 'Inglês']


In [ ]:
df["idade"] = df["ano_base"] - df["Ano nasc"]

In [ ]:
features_excluir_negocio = [
    "RA",                # identificador
    "Defas",             # origem da variável alvo (data leakage)
    "Destaque IEG",      # texto livre (NLP futuro)
    "Destaque IDA",
    "Destaque IPV"
]
features_excluir = list(
    set(features_excluir_missing + features_excluir_negocio)
)

print("\nFeatures excluídas:")
print(features_excluir)

df_eda = df.drop(columns=features_excluir, errors="ignore")

In [ ]:
# Critério técnico:
# 10% a 40%   -> avaliar

features_missing = missing.loc[
    missing["pct_missing"].between(0.10, 0.40), "feature"
].tolist()
print("\nFeatures a excluir por alto percentual de missing:", features_missing)

In [ ]:
numericas = df_eda.select_dtypes(include=["int64", "float64"]).columns.tolist()
numericas.remove("ano_base") 

desc = df_eda[numericas].describe().T
desc


In [ ]:
target_map = {"Em fase": 2, "Moderada": 1, "Severa": 0}
y_num = df_eda["classe_defas"].map(target_map)

corrs = (
    df_eda[numericas]
    .corrwith(y_num)
    .sort_values(key=abs, ascending=False)
)
corrs

In [ ]:
categoricas = df_eda.select_dtypes(include=["object"]).columns.tolist()
categoricas.remove("classe_defas")

cardinalidade = df_eda[categoricas].nunique().sort_values(ascending=False)
cardinalidade


In [ ]:
# Critério técnico:
# Alta cardinalidade (>30) → avaliar agrupamento ou exclusão
alta_cardinalidade = cardinalidade[cardinalidade > 30].index.tolist()

In [ ]:
# INFORMAÇÃO MÚTUA (IMPORTÂNCIA ESTATÍSTICA)
X_num = df_eda[numericas].fillna(df_eda[numericas].median())
X_cat = pd.get_dummies(
    df_eda[categoricas],
    drop_first=True,
    dummy_na=True
)

X = pd.concat([X_num, X_cat], axis=1)

mi = mutual_info_classif(X, y_num, random_state=42)
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)

mi_scores.head()

In [ ]:
features_finais = [
    # Numéricas
    "idade",
    "INDE", "IAN", "IDA", "IEG", "IAA", "IPS", "IPV",
    "Matem", "Portug", "Inglês",
    "Nº Av",
    "Cg", "Cf", "Ct",

    # Categóricas
    "Gênero",
    "Instituicao_padronizada",
    "Fase",
    "Pedra",

    # Binárias
    "Indicado",
    "Atingiu PV",
    "Rec_Psicologia_padronizada"
]

features_finais = [f for f in features_finais if f in df_eda.columns]
print(features_finais)

In [ ]:
X_final = df_eda[features_finais]
y_final = df_eda["classe_defas"]

print("Shape final X:", X_final.shape)
print("Distribuição final y:")
print(y_final.value_counts(normalize=True))

In [ ]:
X_final.head()

In [ ]:
X_final.isna().sum()

In [ ]:
X_final.loc[: , "classe_defas"] = y_final

X_final.to_parquet("../data/processed/dados_para_modelagem.parquet", index=False)